In [1]:
from transformers import AutoTokenizer 
from datasets import load_dataset

/Users/naowak/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Cedille/fr-boris")

In [3]:
tokenizer("Salut les gens", return_tensors="pt")

{'input_ids': tensor([[19221,   315, 10287,   308,   641]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [4]:
datasets = load_dataset('csv', data_files='./train.csv')

Using custom data configuration default-e22b2d9fa03e2cfd
Found cached dataset csv (/Users/naowak/.cache/huggingface/datasets/csv/default-e22b2d9fa03e2cfd/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 1/1 [00:00<00:00, 111.94it/s]


In [6]:
datasets['train']

Dataset({
    features: ['text'],
    num_rows: 1
})

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

100%|██████████| 1/1 [00:21<00:00, 21.33s/ba]


In [14]:
len(tokenized_datasets['train'][0]['input_ids'])

5089690

In [17]:
block_size = 1024

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {
        k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i: i + block_size]
            for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
)

100%|██████████| 1/1 [00:07<00:00,  7.24s/ba]


In [18]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4970
    })
})